In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
my_results_path = "outputs"
sota_results_path = "sota_results"

methods_sota = ["simod", "dsim", "rims"]
my_methods = ["prob", "det"]
casestudies = ["purchasing", "acr", "cvs", "bpi12", "bpi17"]
metrics = ["ngd", "ctd", "car"]
entropy_metrics = ["etd_entropy"]
rule_metrics = ["rule_cfd", "rule_atd", "rule_etd", "rule_wtd"]
depth_range = list(range(6))
best_depth = 5

In [3]:
def load_sota_results():
    data = {}
    for method in methods_sota:
        data[method] = {}
        for case in casestudies:
            path = os.path.join(sota_results_path, method, case, "distances.json")
            if os.path.exists(path):
                with open(path, "r") as f:
                    results = json.load(f)
                for metric in metrics + entropy_metrics:
                    val = np.mean(results.get(metric, []))
                    data[method].setdefault(case, {})[metric] = val
    return data

sota_data = load_sota_results()


In [4]:
def load_my_results():
    data = {method: {} for method in my_methods}
    for method in my_methods:
        for case in casestudies:
            path = os.path.join(my_results_path, case, "distances.json")
            if os.path.exists(path):
                with open(path, "r") as f:
                    results = json.load(f)
                for depth in depth_range:
                    key = f"maxdepth_{depth}"
                    if key in results.get(method, {}):
                        for metric in metrics + entropy_metrics:
                            vals = results[method][key].get(metric, [])
                            mean = np.mean(vals) if vals else np.nan
                            std = np.std(vals) if vals else np.nan
                            data[method].setdefault(case, {}).setdefault(metric, {})[depth] = (mean, std)
    return data

my_data = load_my_results()


In [5]:
def generate_comparison_table(depth):
    rows = []
    for case in casestudies:
        for metric in metrics:
            row = {"Case Study": case, "Metric": metric}

            # SOTA methods
            for method in methods_sota:
                val = sota_data.get(method, {}).get(case, {}).get(metric, np.nan)
                row[method] = val

            # My methods
            row[f"DBPS"] = my_data["prob"].get(case, {}).get(metric, {}).get(depth, (np.nan,))[0]

            rows.append(row)

    df = pd.DataFrame(rows)
    df.set_index(["Case Study", "Metric"], inplace=True)

    # Highlight min values
    styled = df.style.highlight_min(axis=1, color='green').highlight_max(axis=1, color='red')
    return df, styled

df_min, styled_min = generate_comparison_table(best_depth)
print("Accuracy Comparison.")
styled_min


Accuracy Comparison.


In [6]:
print("Avg Accuracy Comparison.")
df_min.groupby("Metric").mean().style.highlight_min(axis=1, color='green')

Avg Accuracy Comparison.


,simod,dsim,rims,DBPS
Metric,,,,
car,232.371577,229.907729,240.393601,200.123127
ctd,356.063472,194.920405,170.534851,153.829981
ngd,0.660645,0.486957,0.484525,0.275177


In [7]:
def generate_comparison_table(depth):
    rows = []
    for case in casestudies:
        for metric in metrics:
            row = {"Case Study": case, "Metric": metric}

            # SOTA methods
            for method in [x for x in methods_sota if x.startswith("simod")]:
                val = sota_data.get(method, {}).get(case, {}).get(metric, np.nan)
                row[method] = val

            # My methods
            row[f"DBPS"] = my_data["prob"].get(case, {}).get(metric, {}).get(depth, (np.nan,))[0]

            rows.append(row)

    df = pd.DataFrame(rows)
    df.set_index(["Case Study", "Metric"], inplace=True)

    # Highlight min values
    styled = df.style.highlight_min(axis=1, color='green')
    return df, styled

df_min, styled_min = generate_comparison_table(best_depth)
print("Accuracy Comparison (White-Box).")
styled_min


Accuracy Comparison (White-Box).


In [8]:
methods = ['simod', 'maxdepth_5']

rows = []

for case in casestudies:
    filename = f'{my_results_path}/rule_distances_{case}.json'
    if os.path.exists(filename):
        with open(filename) as f:
            data = json.load(f)
            for metric in rule_metrics:
                row = {
                    'Case Study': case,
                    'Metric': metric
                }
                for method in methods:
                    row[method] = data.get(metric, {}).get(method, None)
                rows.append(row)
    else:
        print(f"Warning: File {filename} not found")

df = pd.DataFrame(rows)
df.rename(columns={"maxdepth_5": "DBPS"}, inplace=True)
df.set_index(["Case Study", "Metric"], inplace=True)

styled = df.style.highlight_min(axis=1, color='green')
print("Rule Accuracy Comparison (White-Box).")
styled

Rule Accuracy Comparison (White-Box).


In [9]:
print("Avg Rule Accuracy Comparison (White-Box).")
df.groupby("Metric").mean().style.highlight_min(axis=1, color='green')

Avg Rule Accuracy Comparison (White-Box).


,simod,DBPS
Metric,,
rule_atd,182.609877,174.450082
rule_cfd,0.232761,0.069019
rule_etd,32.066359,9.703796
rule_wtd,882.366322,812.444997


In [10]:
def generate_entropy_comparison_table(depth):
    rows = []
    for case in casestudies:
        for metric in entropy_metrics:
            row = {"Case Study": case, "Metric": metric}

            for method in methods_sota:
                val = sota_data.get(method, {}).get(case, {}).get(metric, np.nan)
                row[method] = val

            row[f"prob_DBPS"] = my_data["prob"].get(case, {}).get(metric, {}).get(depth, (np.nan,))[0]
            row[f"det_DBPS"] = my_data["det"].get(case, {}).get(metric, {}).get(depth, (np.nan,))[0]

            rows.append(row)

    df = pd.DataFrame(rows)
    df.set_index(["Case Study", "Metric"], inplace=True)

    styled = df.style.highlight_max(axis=1, color='green').highlight_min(axis=1, color='red')
    return df, styled

df_max, styled = generate_entropy_comparison_table(best_depth)
print("Execution Time Entropy Comparison.")
styled

Execution Time Entropy Comparison.


,,simod,dsim,rims,prob_DBPS,det_DBPS
Case Study,Metric,,,,,
purchasing,etd_entropy,1.331335,1.160079,1.196356,1.380686,0.564997
acr,etd_entropy,1.005804,0.740469,0.593001,1.149228,0.845624
cvs,etd_entropy,0.640277,0.566857,0.575200,0.366053,0.078767
bpi12,etd_entropy,1.654026,0.431299,0.557028,2.245392,0.599924
bpi17,etd_entropy,1.975552,0.603484,0.611401,2.208731,0.758401


In [11]:
df_max.groupby("Metric").mean()

,simod,dsim,rims,prob_DBPS,det_DBPS
Metric,,,,,
etd_entropy,1.321399,0.700438,0.706598,1.470018,0.569543
